# <center>Advanced MPI</center>


### Stencil Computations

Iterative Stencil Loops (ISLs) or Stencil computations are a class of numerical data processing solution which update array elements according to some fixed pattern, called a stencil.They are most commonly found in computer simulations, e.g. for computational fluid dynamics in the context of scientific and engineering applications. Other notable examples include solving partial differential equations, the Jacobi kernel, the Gauss–Seidel method,image processing and cellular automata. 

The regular structure of the arrays sets stencil techniques apart from other modeling methods such as the Finite element method. Most finite difference codes which operate on regular grids can be formulated as ISLs.

SLs perform a sequence of sweeps (called timesteps) through a given array. Generally this is a 2- or 3-dimensional regular grid. The elements of the arrays are often referred to as cells. In each timestep, all array elements are updated. Using neighboring array elements in a fixed pattern (the stencil), each cell's new value is computed. In most cases boundary values are left unchanged, but in some cases (e.g. LBM codes) those need to be adjusted during the computation as well. Since the stencil is the same for each element, the pattern of data accesses is repeated. Source [Wikipedia](https://en.wikipedia.org/wiki/Iterative_Stencil_Loops)

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e8/Von_Neumann_neighborhood.svg/800px-Von_Neumann_neighborhood.svg.png" height=200px width=200px>

## Basic Stencil Operation


![Basic Stencil Operation](./img/stencil_basic.png)

# How to run MPI programs for the stencil codes 

### Step 1: Create and initialize the Grid 

```c++
#define N 10  // Grid size
#define ITERATIONS 100  // Number of iterations

// Function to initialize the grid
void initialize_grid(double grid[N][N]) {
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            grid[i][j] = rand() % 100;
        }
    }
}
```

### Step 2: Define the stencil Step operation

```c++
// Function to perform the stencil computation
void stencil_step(double grid[N][N], double new_grid[N][N]) {
    for (int i = 1; i < N-1; i++) {
        for (int j = 1; j < N-1; j++) {
            new_grid[i][j] = 0.25 * (grid[i-1][j] + grid[i+1][j] + grid[i][j-1] + grid[i][j+1]);
        }
    }
}
```



### Step 3: Define how to copy the Grid 
```c++
// Function to copy the new grid to the old grid
void copy_grid(double dest[N][N], double src[N][N]) {
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            dest[i][j] = src[i][j];
        }
    }
}

```


### Step 4: Create the MPI process for running the stencil operation over the ranks

```c++
int main(int argc, char *argv[]) {
    int rank, size;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    double grid[N][N], new_grid[N][N];
    initialize_grid(grid);

    for (int iter = 0; iter < ITERATIONS; iter++) {
        stencil_step(grid, new_grid);
        copy_grid(grid, new_grid);

        // Exchange boundary rows between neighboring processes
        if (rank > 0) {
            MPI_Send(grid[1], N, MPI_DOUBLE, rank-1, 0, MPI_COMM_WORLD);
            MPI_Recv(grid[0], N, MPI_DOUBLE, rank-1, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        }
        if (rank < size-1) {
            MPI_Send(grid[N-2], N, MPI_DOUBLE, rank+1, 0, MPI_COMM_WORLD);
            MPI_Recv(grid[N-1], N, MPI_DOUBLE, rank+1, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        }
    }

    MPI_Finalize();
    return 0;
}
```

In [ ]:
!mpicc code/basic_stencil.cpp -o basic_stencil.exe

#!srun -N 1 ./basic_stencil.exe
#!ulimit -s 800000; srun -N 1 ./basic_stencil.exe